# Pre-Requisites

First off we install the libraries, login to HuggingFaceHub and then import those libraries.

In [1]:
# @title Installing required libraries

!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers
!pip -q install langchain
!pip -q install peft chromadb
!pip -q install unstructured
!pip install -q sentence_transformers
!pip install -q faiss-gpu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.

In [2]:
# @title Displaying GPU information

!nvidia-smi

Mon Sep 18 12:18:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# @title Logging in to HuggingFace

from huggingface_hub import login

login()

In [4]:
# @title Importing the libraries

import nltk
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Quantization of the Model

## What is Quantization?
It's like compressing your model to make it smaller, faster and more-power efficient.

## Why Quantization?
1. **Smaller Size**: Easier to deploy
2. **Faster**: Quicker predictions
3. **Less Power**: Saves battery


In [5]:
# @title Defining the config for the quantization

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                            # --> Load weights in 4-bit to save space.
    bnb_4bit_quant_type="nf4",                    # --> Type of 4-bit quantization.
    bnb_4bit_compute_dtype=torch.bfloat16,        # --> Using torch.bfloat16 for faster calculations.
    bnb_4bit_use_double_quant=False               # --> False means no double compression.
)

# LLAMA2-13B

We now set up the Large Language Model (LLM), i.e, Meta's LLama2-13b chat version. First we define a Tokenizer that tokenizes plaintext into tokens. Then we set up the model by passing our quantized config (which we defined earlier), and mapping it to use GPU as the device.

In [6]:
# @title Tokenizer and Model

id = "meta-llama/Llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(id)
model = AutoModelForCausalLM.from_pretrained(id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Prompt Engineering

## Constant Declaration
It is required to define some constant denotations in order to structure the user input prompt correctly :

1. **B_INST**, **E_INST** => Marks the beginning and the end of the Instructions
2. **B_SYS**, **E_SYS** => Marks the beginning and the end of the System Prompts
3. **DEFAULT_SYSTEM_PROMPT** => Default behavior for the LLM System

## Prompt Structuring
We define a function to build and return a well-structured prompt that uses the aforementioned constants. This is done so that the LLM understands the context fully before attempting to respond to the user's raw queries.

In [7]:
# @title Prompt Template Configuration

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Your primary job is to always answer as helpfully as possible, \
while being safe. Your responses must not contain anything harmful, unethical, racist, sexist, toxic, dangerous, \
or illegal NSFW content. You must always ensure that your responses are socially, politically, morally and overall \
unambiguous, unbiased and positive in nature. If a question does not make sense, or is not factually coherent, explain \
why instead of answering something incorrect. Avoid spreading misinformation as a result of hallucination at any cost. \
If you do not know the answer to any question, be honest by clearing that up and letting the user know so.
"""

def get_prompt(instruction, new_sys_prompt=DEFAULT_SYSTEM_PROMPT):
  SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + new_sys_prompt + E_SYS
  prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
  return prompt_template

# Vector Stores and Document Processing

In this step, we focus on embedding the user document (CSV) into vectors, and storing them into a Vector Database for faster retrieval. Such an approach is also known as **Retrieval-Augmented Generation (RAG)**.

In [8]:
# @title Importing Langchain and its Methods

from langchain.document_loaders.csv_loader import CSVLoader # to load the document as plaintext
from langchain.text_splitter import RecursiveCharacterTextSplitter  # to split the plaintext into chunks
from langchain.embeddings import HuggingFaceEmbeddings  # to embed chunks into vectors
from langchain.vectorstores import FAISS  # to store those vectors in a database

In [10]:
# @title Loading the CSV file into a CSV Loader

# I'm using a Car Resale Prices csv file, but you are free to use any CSV that you desire to.
# This CSV will be available in this repo itself, if you need.
loader = CSVLoader("/content/car_resale_prices.csv", encoding="utf-8", csv_args={
    'delimiter': ','
})
data = loader.load()
data[:1]

[Document(page_content=': 0\nfull_name: 2017 Maruti Baleno 1.2 Alpha\nresale_price: ₹ 5.45 Lakh\nregistered_year: 2017\nengine_capacity: 1197 cc\ninsurance: Third Party insurance\ntransmission_type: Manual\nkms_driven: 40,000 Kms\nowner_type: First Owner\nfuel_type: Petrol\nmax_power: 83.1bhp\nseats: 5\nmileage: 21.4 kmpl\nbody_type: Hatchback\ncity: Agra', metadata={'source': '/content/car_resale_prices.csv', 'row': 0})]

In [11]:
# @title Generating Embeddings for the data

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
# type(embeddings)

In [12]:
# @title Storing embeddings into a Vector Database

DB_FAISS_PATH = "vectorstore/db_faiss"

db = FAISS.from_documents(data, embeddings)
db.save_local(DB_FAISS_PATH)

# Pipeline and Bot Creation

This stage defines a function `create_pipeline()` to set up a text-generation pipeline using HuggingFace libraries, and a `ChatBot` class to create a chatbot. The class constructor initializes the chatbot's memory, prompt, and data retriever. The `create_chat_bot()` method within the class combines these elements to generate a chatbot that can both retrieve relevant information and generate text based on that. It returns this combined chatbot for use.

In [13]:
# @title Importing the necessary libraries

from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory

In [14]:
# @title Chatbot memory configuration

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",   # specifies that the memory will be used to store chat history.
    key=5,   # memory will keep track of the last 5 interactions in the conversation.
    return_messages=True   # indicates that the stored messages can be returned for future use.
)

In [15]:
# @title Defining a retriever object for the database

retriever = db.as_retriever()

In [16]:
# @title Pipeline Generation

def create_pipeline(max_new_tokens=512):
    pipe = pipeline("text-generation",
                model=model,
                tokenizer = tokenizer,
                max_new_tokens = max_new_tokens,
                temperature = 0.1)   # You can set the temperature to any value you desire, but make sure it's a strictly positive float.
    return pipe

In [17]:
# @title Encapsulating the process into a class

class ChatBot:
  def __init__(self, memory, prompt, task:str = "text-generation", retriever = retriever):
    self.memory = memory
    self.prompt = prompt
    self.retriever = retriever

  def create_chat_bot(self, max_new_tokens = 512):
    hf_pipe = create_pipeline(max_new_tokens)
    llm = HuggingFacePipeline(pipeline =hf_pipe)
    qa = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=self.retriever,
      memory=self.memory,
      combine_docs_chain_kwargs={"prompt": self.prompt}
  )
    return qa

# Setting up the Inference

Now we create a CSV bot and a prompt template for the same.

In [18]:
# @title Prompt Template

from langchain import PromptTemplate

instruction = "Given the context that has been provided : \n{context}\nAnswer the following question : \n{question}"

system_prompt = """
You are an expert of car sales, you are fully aware of the current scenario in car prices. \
Your job is to respond to the queries that have been asked to you, in a polite and \
instructive manner, referring the data that's been given to you.responses must be  \
extremely short and accurate, do not write multiple huge paragraphs. In case you do not \
know the answer to a particular question, simply clarify the fact that you don't know, \
avoid spreading misinformation at any cost. Maximum length of your responses should be 2-3 sentences.
"""

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

[INST]<<SYS>>

You are a helpful, respectful and honest assistant. Your primary job is to always answer as helpfully as possible, while being safe. Your responses must not contain anything harmful, unethical, racist, sexist, toxic, dangerous, or illegal NSFW content. You must always ensure that your responses are socially, politically, morally and overall unambiguous, unbiased and positive in nature. If a question does not make sense, or is not factually coherent, explain why instead of answering something incorrect. Avoid spreading misinformation as a result of hallucination at any cost. If you do not know the answer to any question, be honest by clearing that up and letting the user know so.

You are an expert of car sales, you are fully aware of the current scenario in car prices. Your job is to respond to the queries that have been asked to you, in a polite and instructive manner, referring the data that's been given to you.responses must be  extremely short and accurate, do not wr

In [19]:
# @title Bot Creation

car_bot_instance = ChatBot(memory=memory, prompt=prompt)
car_bot = car_bot_instance.create_chat_bot()

# Inference

We finally now test the chatbot that we created.

In [20]:
# @title Creating a reply function
from IPython.display import display, HTML

def reply(message):
  bot_response = car_bot({"question": message})['answer']
  display(HTML(bot_response))

In [21]:
# @title Testing the bot

reply("What's the mileage of a 2018 Tata Hexa XTA as shown in the data?")

In [25]:
reply("What is the engine capacity of a 2011 Toyota Etios VX as shown in the data?")

In [29]:
reply("List the different body types as shown in the data.")

In [30]:
reply("In the Sedan body type, which is the best car with respect to the engine capacity?")

In [31]:
reply("How does the Fuel Type impact the Max Power? Deduce a pattern.")

In [35]:
final_prompt = """
Suppose my showroom sold 3 cars :
First car = 2022 Mercedes-Benz AMG A 35 4MATIC BSVI
Second car = 2011 Chevrolet Beat LT
Third car = 2014 Audi Q3 35 TDI Quattro Premium Plus
Calculate the average resale price of these 3 cars.
"""

reply(final_prompt)